# Convolutional Neural Network (1x1/2 width/strides, 8 filters; 5x5/2 width/strides, 32 filters)
To run this notebook, install the [hyperas](https://github.com/maxpumperla/hyperas) dependency and run each cell. This notebook trains the feed forward neural network on the training data, and evaluates it on the test dataset. It saves the model into the `\weights` directory.

In [ ]:
# Install dependencies
%pip install hyperas

In [2]:
import h5py
from os.path import join,exists
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, Flatten, Dense, MaxPool2D, Dropout
from tensorflow.keras.optimizers import RMSprop
from hyperas.distributions import choice
from hyperas import optim
from keras.callbacks import ModelCheckpoint
from keras.constraints import maxnorm
from random import randint
from keras import backend as K
import os
import numpy as np
from sklearn.utils import shuffle
from hyperopt import Trials, STATUS_OK, tpe
import tensorflow as tf
K.set_image_data_format('channels_first')

In [3]:
# Changes to working directory
os.chdir('/content/drive/My Drive/github/NNanobody/regression')

In [4]:
def create_model(X_train, Y_train, X_test, Y_test):
    """
    create_model compiles and fits the model for five epochs in order for tuning 
    of hyperparameters.

    :param X_train: Independent variable of training data (one hot encoded CDR3 sequences)
    :param Y_train: Dependent variable of training data (enrichment of CDR3 sequences)
    :param X_test: Independent variable of test data (one hot encoded CDR3 sequences)
    :param Y_test: Dependent variable of test data (enrichment of CDR3 sequences)
    :return: Returns the validation loss, hyperopt status, and the model itself after training
    """ 
    W_maxnorm = 3 # Normalization of weights to prevent overfit
    DROPOUT = {{choice([0.3,0.5,0.7])}} # Dropout choices for hyperparameters optimization

    # Construct CNN. Uses leaky ReLU activation function to fix "dying ReLU" problem and speed up training.
    model = Sequential()
    model.add(Conv2D(8, (1, 1), padding='same', input_shape=(20, 1, 20),activation='leaky_relu',kernel_constraint=maxnorm(W_maxnorm)))
    model.add(MaxPool2D(pool_size=(1, 1),strides=(1,1)))
    model.add(Conv2D(64, (1, 5), padding='same', activation='leaky_relu',kernel_constraint=maxnorm(W_maxnorm)))
    model.add(MaxPool2D(pool_size=(1, 2),strides=(1,2)))
    model.add(Flatten())
    model.add(Dense(16,activation='leaky_relu',kernel_constraint=maxnorm(W_maxnorm)))
    model.add(Dropout(DROPOUT))
    model.add(Dense(1))
    
    myoptimizer = RMSprop(lr={{choice([0.01,0.001,0.0001,1e-5,1e-1])}}, rho=0.9, epsilon=1e-06) # RMSProp optimizer for the model. Learning rate choices for hyperparameter optimizaiton.
    mylossfunc='mean_squared_error'
    model.compile(loss=mylossfunc, optimizer=myoptimizer) # Compile model
    result = model.fit(X_train, Y_train, batch_size=100, epochs=5,validation_split=0.1,verbose=False) # Train model for 5 epochs (hyperparameter optimization)

    val_loss = np.amax(result.history['val_loss']) # Save validation loss to return

    return {'loss': val_loss, 'status': STATUS_OK, 'model': model}

In [16]:
def data():
  """
  data is a helper function that returns the data necessary for training and validation.

  :return: Returns the shuffled training data and testing data
  """
  
  data_train = h5py.File('./data/Hold out Top 4%/train.h5.batch1', 'r') # Load embedded data from .h5 file
  X_train = np.array(data_train['data'])
  Y_train = np.array(data_train['label'])
  X_train_shuffled, Y_train_shuffled = shuffle(X_train, Y_train) # Shuffle training data

  data_test = h5py.File('./data/Test set Regression/test.h5.batch1', 'r')
  X_test = np.array(data_test['data'])
  Y_test = np.array(data_test['label'])
  return X_train_shuffled, Y_train_shuffled, X_test, Y_test

In [17]:
# Hyperparameter optimization according to the hyperas documentation. Evaluates 10 times with the tpe suggested search method.
best_run, best_model = optim.minimize(model=create_model,
                                          data=data,
                                          algo=tpe.suggest,
                                          max_evals=10,
                                          trials=Trials(),
                                          notebook_name='seq_32_32',
                                          verbose=False)

  0%|          | 0/10 [00:00<?, ?it/s, best loss: ?]

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)



100%|██████████| 10/10 [01:49<00:00, 10.94s/it, best loss: 0.2794625461101532]


In [18]:
X_train, Y_train, X_test, Y_test = data() # Load all data
best_model.fit(X_train, Y_train, batch_size=100, epochs=20,validation_split=0.1,verbose=True) # Train the hyperparameter optimized model

Epoch 1/20
586/586 [==============================] - 2s 4ms/step - loss: 0.2507 - val_loss: 0.2398
Epoch 2/20
586/586 [==============================] - 2s 4ms/step - loss: 0.2471 - val_loss: 0.2435
Epoch 3/20
586/586 [==============================] - 2s 4ms/step - loss: 0.2460 - val_loss: 0.2444
Epoch 4/20
586/586 [==============================] - 2s 4ms/step - loss: 0.2442 - val_loss: 0.2367
Epoch 5/20
586/586 [==============================] - 2s 4ms/step - loss: 0.2429 - val_loss: 0.2381
Epoch 6/20
586/586 [==============================] - 2s 4ms/step - loss: 0.2413 - val_loss: 0.2377
Epoch 7/20
586/586 [==============================] - 2s 4ms/step - loss: 0.2405 - val_loss: 0.2353
Epoch 8/20
586/586 [==============================] - 2s 4ms/step - loss: 0.2404 - val_loss: 0.2382
Epoch 9/20
586/586 [==============================] - 2s 4ms/step - loss: 0.2383 - val_loss: 0.2378
Epoch 10/20
586/586 [==============================] - 2s 4ms/step - loss: 0.2376 - val_loss: 0.2367

In [19]:
best_model.evaluate(X_test, Y_test) # Evaluate model on the test set

710/710 [==============================] - 1s 2ms/step - loss: 0.4491


0.44907253980636597

In [20]:
best_model.save('../weights/regression/Hold out Top 4%/seq_32_32') # Save the model

INFO:tensorflow:Assets written to: ../weights/regression/Hold out Top 4%/seq_emb_32x1_16/assets
